<a href="https://colab.research.google.com/github/kreshuklab/teaching-dl-course-2019/blob/master/Webinars/exercise1/image_manipulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Manipulation Tutorial

## Downloading the data

We are going to work with Kaggle 2018 Data Science Bowl data.
To start with go the [data webpage](https://www.kaggle.com/c/data-science-bowl-2018) and read the data description.

Now let's download the data. To make it easier, we're going to work with a subset of it. 

In [0]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1O66UElt2ZfhLXUKKX_nTxmIXh6fMA2rT' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1O66UElt2ZfhLXUKKX_nTxmIXh6fMA2rT" -O kaggle_data.zip && rm -rf /tmp/cookies.txt

Remember that you can execute any bash command from the Notebook if you preceed the command name with '!'.

And please check whether the downloaded archive is around 80M (the value after the progress bar [ <=> ]). If the value is much smaller, rerun the previous cell - probably something failed. 

Those of you who like bash can play around with unzipping the data into nice folders. The rest of you can just run the following:


In [0]:
!unzip -qq kaggle_data.zip && rm kaggle_data.zip && rm stage1_test.zip
!mkdir nuclei_data && unzip -qq stage1_train.zip -d nuclei_data/ && rm stage1_train.zip

Don't forget that you can always check what is happening in your directory using `ls` :

In [0]:
!ls

In [0]:
!ls nuclei_data

Wow, that was a loooot of folders. Hint: you can clear the output of the cell by clicking the 'clear output' button below the 'run cell'.

Now let's check what they contain by taking one random folder name:




In [0]:
!ls nuclei_data/eb1df8ed879d04b36980b0958a0e8fc446ad08c0bdcf3b5f42e3db023187c7e5

In [0]:
!ls nuclei_data/eb1df8ed879d04b36980b0958a0e8fc446ad08c0bdcf3b5f42e3db023187c7e5/images

Okay, this one contains a png image. __TASK:__ Check other random folder names to make sure the structure is the same.

## Displaying images

Now we want to load some pictures and look at them. For this we would need the following libraries:

In [0]:
# we want to show images directly in the notebook
%matplotlib inline
import os       # to list folders content
import numpy as np    # scientific computing 
import matplotlib.pyplot as plt   # plotting and visualisation
import cv2    # computer vision library, works with images as numpy arrays
from google.colab.patches import cv2_imshow   # the native cv2 function imshow doesn't work in google colab 

In [0]:
image_path = 'nuclei_data/eb1df8ed879d04b36980b0958a0e8fc446ad08c0bdcf3b5f42e3db023187c7e5/images/eb1df8ed879d04b36980b0958a0e8fc446ad08c0bdcf3b5f42e3db023187c7e5.png'
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

In [0]:
cv2_imshow(image)

In [0]:
# Another way to show loaded images that is more flexible than cv2_imshow is using plt from matplotlib
plt.imshow(image)    # Hint: if you don't like the colormap change it by setting cmap to gray 

If we want to have a better overview of what is happening in the folder, loading the images one by one is not the best approach. What we will do now is list all the folders we have, and write a function that will load an image file from a random folder.

In [0]:
folders_list = os.listdir('nuclei_data')  # get the list of all the folders inside nuclei_data
folders_list[0]   # let's see how the folder names look like

In [0]:
def show_random_image(directory):
  rand_idx = np.random.randint(0, len(directory))   # get a random index
  img_path = # TASK: what would be the image path here?
  random_image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
  cv2_imshow(random_image)

In [0]:
# Now test your function and see how the images in your folder look like
show_random_image(directory)

## Operations on images

Now let's look more into the operations that you can perform on your image. Firstly, let's write a function that visualises two images at the same time to examine the transformations visually.

In [0]:
# This is something we would need matplotlib for
def show_two_images(image1, image2):
    f, axarr = plt.subplots(1, 2)   # we need two images in a row
    axarr[0].imshow(image1, cmap='gray')
    axarr[1].imshow(image2, cmap='gray')
    _ = [ax.axis('off') for ax in axarr]   # remove the axis ticks
    plt.show()

In [0]:
# Let's check how this looks like
show_two_images(image, image[:100, :100])

Looks fine. Now let's go through the transforms offered us by cv2 library.

## Resizing image

In [0]:
# fx and fy are the scale factors along x and y axes 
resized_image = cv2.resize(image, None, fx=2, fy=2, interpolation=cv2.INTER_LINEAR)
cv2_imshow(resized_image)

TASK : try different way of [interpolation](https://docs.opencv.org/2.4/modules/imgproc/doc/geometric_transformations.html#cv.Resize). Which one looks better visually: nearest neighbor, bilinear or bicubic?
Hint: use `show_two_images` function to visualise the difference. 

Advanced TASK: modify `show_two_images` to show n images - given a list of any number of images the function should plot all of them in a row. Visualise multiple interpolations at the same time.

## Translating image

In [0]:
rows,cols = image.shape
# we need to define a transfromation matrix for translation
# here we shift the image 100 pixels on the horizontal axis and 50 on vertical
M = np.float32([[1, 0, 100],[0, 1, 50]])
translated_image = cv2.warpAffine(image, M, (cols, rows))
show_two_images(image, translated_image)

## Rotating image 

TASK: follow the instructions [here](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_geometric_transformations/py_geometric_transformations.html#rotation) to rotate your image and visualise it.

Advanced TASK: write a function that given an image and a number N will 
* rotate the image by a random angle in range (0, N) degrees
* translate the image by a random number of pixels horizontally and vertically, where the number of pixels is not bigger than 1/N of image dimentions (e.g., for image shape = 100*200 and N = 2 the max translation is 50 and 100 pixels vertically and horizontally, respectively)

In [0]:
# Your code here
rotated_image = #  Your code here
show_two_images(image, rotated_image)

## Affine Transformation

In [0]:
# for the affine transformation we need to define three points 
# in the input image and their desired locations after the transformation
points1 = np.float32([[50, 50], [200, 50], [50, 200]])
points2 = np.float32([[10, 100], [200, 50], [100, 250]])

M = cv2.getAffineTransform(points1, points2)

affine_transformed_image = cv2.warpAffine(image, M, (cols, rows))
show_two_images(image, affine_transformed_image)

TASK: flip your image using affine transformation. Note, that image.shape would show you the size as (rows, cols) while the points should be be provided in (cols, rows) format.

Hint: for the points1 take the coordinates of the corner pixels. Where do you want these points to end up if you flip the image? 

Advanced TASK: apply Perspective Transformation on your image as described [here](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_geometric_transformations/py_geometric_transformations.html#perspective-transformation).

## Thresholding image
In simple cases you can get an object mask (segment your object) using plain thresholding. Let's see how good it works for our images.

In [0]:
# pixels below the threshold (background) will be set to 0
# pixels above or equal to the threshold (foreground) will be set to the given foreground value, typically 1
threshold = 150
foreground = 1 # value of foreground pixels in resulting binary image
inverted_image = (image.max() - image) # in our example image the objects are dark, thus we invert before we threshold
_, binary_thresholded_image = cv2.threshold(inverted_image, threshold, foreground, cv2.THRESH_BINARY)
show_two_images(inverted_image, binary_thresholded_image)

TASK: try different threshold values. Also compare it to [adaptive thresholding](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_thresholding/py_thresholding.html#adaptive-thresholding). Which one seems to work better? Why? 

## Morphological Transformations
The masks we got out of tresholding look suboptimal. There is noise, holes in the masks and some masks are merged together. We can try to alleviate there problems with such morphological transformations as opening - [errosion](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_morphological_ops/py_morphological_ops.html#erosion) followed by [dilation](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_morphological_ops/py_morphological_ops.html#dilation), and closing - dilation followed by errosion.

Let's see how it looks like.

In [0]:
# we need to set a kernel
# follow the links above to understand what the kernel does
kernel = np.ones((3,3),np.uint8)
opened_image = cv2.morphologyEx(binary_thresholded_image, cv2.MORPH_OPEN, kernel, iterations=3)
show_two_images(binary_thresholded_image, opened_image)


TASK: try different kernel sizes and number of iterations. How does it affect the segmentation masks? 

Try closing (set the operation type to cv2.MORPH_CLOSE). Try different parameters as well.
## Finding edges 
Image gradients can be used to detect object edges. Let's try to use the [Canny algorithm](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_canny/py_canny.html#canny-edge-detection) from cv2 library. 

In [0]:
# read more about minVal and maxVal arguments in the link above
edges = cv2.Canny(image, 100, 150)
show_two_images(image, edges)

TASK: try different minVal and maxVal. Does any combination give perfect object boundaries?

Advanced TASK: segment the image with the watershed algorithm as described [here](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_watershed/py_watershed.html#image-segmentation-with-watershed-algorithm).

## Further reading:
Take a look at the [cv2 tutorials](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_table_of_contents_imgproc/py_table_of_contents_imgproc.html#image-processing-in-opencv) to find functions that might be useful for you. 
